In [2]:
# Pipeline 2 description
# Fit a beta distribution to all the tissue columns w/o the nan values in the training set.
# Use the beta distributions conditioned on the location to fill the NaN values.
# Once you have finished filling in the NaNs, your training set will be complete
# To fill in the values in the test set, take the test vector's values, locations and find the
# top x distributions that the vector is most likely to come from.
# Use these top x as the training set for a regressor that will be used to predict the missing values in the training
# set.

In [3]:
# load imports
import math
import numpy as np 
from IPython.display import Image  
import pandas
import os
from scipy.stats import beta
# Enter your directory here
os.chdir('/Users/abkhanna/Documents/workspace/Princeton/cos424/a2/methylation_imputation-master')

In [4]:
# Load the data
train_bed = pandas.read_csv('data/intersected_final_chr1_cutoff_20_train.bed', sep='\t', header=None)
sample_p_bed = pandas.read_csv('data/intersected_final_chr1_cutoff_20_sample_partial.bed', sep='\t', header=None)
sample_f_bed = pandas.read_csv('data/intersected_final_chr1_cutoff_20_sample_full.bed', sep='\t', header=None)

In [5]:
# fit the distributions
# first trial with just a single tissue
tissue_sample = train_bed[4]
tissue_sample_list = tissue_sample
# remove NaNs
tissue_sample_no_nans = [x for x in tissue_sample_list if ~np.isnan(x)]
a, b, loc, scale = beta.fit(tissue_sample_no_nans)

In [6]:
# build the beta for the test vector
# use the beta to generate the values for the missing values.
beta_gen = np.random.beta
tissue_sample_nan_rep = [x if ~np.isnan(x) else beta_gen(a, b) for x in tissue_sample_list]

In [7]:
# now time to do it with the entire set!
def fill_col(col):
    col_sample_list = col
    col_sample_no_nans = [x for x in col_sample_list if ~np.isnan(x)]
    a, b, loc, scale = beta.fit(col_sample_no_nans)
    beta_gen = np.random.beta
    col_sample_nan_rep = [x if ~np.isnan(x) else beta_gen(a, b) for x in col_sample_list]
    return col_sample_nan_rep

table1 = train_bed[list(range(4,37))]
table2 = []
for i in range(4, 37):
    table2.append(fill_col(table1[i]))
table2 = pandas.DataFrame(table2).transpose()


/Users/abkhanna/Documents/workspace/Princeton/cos424/a2/lib/python2.7/site-packages/scipy/optimize/minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [8]:
table2[32]

0         0.437500
1         0.660377
2         0.852459
3         0.974683
4         0.906250
5         0.777778
6         0.750000
7         0.521739
8         0.875000
9         0.820513
10        0.544304
11        0.833333
12        0.863158
13        0.822222
14        0.529412
15        0.835165
16        0.613636
17        0.892857
18        0.860759
19        0.404255
20        0.750000
21        0.592593
22        0.358974
23        0.239130
24        0.122449
25        0.393443
26        0.316667
27        0.796610
28        0.652174
29        0.437500
            ...   
379521    0.768293
379522    0.848837
379523    0.772727
379524    0.812500
379525    0.585106
379526    0.830000
379527    0.676768
379528    0.647059
379529    0.747368
379530    0.890244
379531    0.564103
379532    0.851351
379533    0.698413
379534    0.846154
379535    0.621212
379536    0.804878
379537    0.703704
379538    0.703704
379539    0.896104
379540    0.847458
379541    0.904762
379542    0.

In [9]:
# Now create a list of (a,b) pairs so you can figure out which distribution is the most likely 
# for your similarity metric
# table2 has all the NaN values filled in in the training set.
ab_tuples = []
def get_ab(col):
    a, b, loc, scale = beta.fit(col)
    return a,b

for i in range(0,32):
    ab_tuples.append(get_ab(table2[i]))

print len(ab_tuples)

32


In [10]:
# once we have the list of ab tuples, we can use them to calculate the probability
# that the test vector came from any one of these distributions
test_vector_nans = sample_p_bed[4] # 4th column has the data, others are informatory
test_vector_no_nans = [x for x in test_vector_nans if ~np.isnan(x)]
similarity_scores = []
i = 0
for a, b in ab_tuples:
    running_product = 0
    for meth in test_vector_no_nans:
        assert (0 < meth < 1)
        probability = beta.logpdf(meth, a, b)
        running_product = running_product + probability
    similarity_scores.append((running_product, i))
    i = i + 1

# sort the scores by the first tuple value
sorted_similarity = sorted(similarity_scores, key=lambda tup: tup[0])
print len(sorted_similarity)

32


In [11]:
# take the top x number of these vectors, and use their values to create a regression.
top_x = 3

y1 = table2[sorted_similarity[-1][1]].as_matrix().tolist()
# y2 = table2[sorted_similarity[-2][1]].as_matrix().tolist()
# y3 = table2[sorted_similarity[-3][1]].as_matrix().tolist()
# print len(y1)
# print len(y2)
# print len(y3)
xs = train_bed[1].as_matrix().tolist()
X = xs # + xs + xs
print "X Len: {0}".format(len(X))
print "xs len: {0}".format(len(xs))
Y = y1 # + y2 + y3
print "Y len: {0}".format(len(Y))
X = np.array(X).reshape(-1,1)
print "X shape: {0}".format(X.shape)
Y = np.array(Y)
print "Y shape: {0}".format(Y.shape)
print "Done!"


X Len: 379551
xs len: 379551
Y len: 379551
X shape: (379551, 1)
Y shape: (379551,)
Done!


In [12]:
# define the window_size
window_size = 50

In [ ]:
# split the elements of X and Y into chunks of 5,000 to prevent fitting time complexity issues
# len of x = len of y
X_partials_array = []
Y_partials_array = []
for i in range(0,len(X), window_size):
    X_partials_array.append(X[i:i+window_size])
    Y_partials_array.append(Y[i:i+window_size])


In [ ]:
# http://datascience.stackexchange.com/questions/989/
# ... svm-using-scikit-learn-runs-endlessly-and-never-completes-execution
# suggests subsampling the data set rather than using the entirety of it.
from sklearn.svm import SVR
svrs = [] # array of svr kernels split across every 5000 rows
for i in range(0, len(X_partials_array)):
    svr_rbf = SVR(kernel='poly', C=1e3, gamma=0.01, cache_size=7000)
    svr_rbf.fit(X_partials_array[i], Y_partials_array[i])
    svrs.append(svr_rbf)


In [ ]:
# from these svrs, predict the missing values in the test vector!
test_tissue = sample_p_bed[4]
nan_vector = sample_p_bed[5]
test_loc_vector = sample_p_bed[1]
filled_test_tissue = []
predict_count = 0
orc = 0

def predict(i):
    svr_bucket = int(i / window_size) # window_size is defined a few cells above
    svr_regressor = svrs[svr_bucket]
    y_predict = svr_regressor.predict(test_loc_vector[i])
#     over_range_count = 0
    if y_predict > 1:
#         over_range_count = over_range_count + 1
        return [0.45], 1
    if y_predict < -1:
        return [0.45], 1
    return y_predict, over_range_count

for i in range(len(nan_vector)):
    if nan_vector[i] == 0 or np.isnan(test_tissue[i]):
        # fill it in!!!
        predict_count = predict_count + 1
        y_predict, n_orc = predict(i)
        orc = orc + n_orc
#         assert 0 < y_predict[0] < 1
        filled_test_tissue.append(y_predict[0])
    elif nan_vector[i] == 1:
        # value is valid! just use it
        y_actual = test_tissue[i]
        assert 0 < y_actual < 1
        filled_test_tissue.append(y_actual)
    else:
        raise ValueError('Invalid state in nan_vector')

print "Filled_test_tissue: {0}".format(len(filled_test_tissue))
print "Test Tissue: {0}".format(len(test_tissue))
print "Predict Count: {0}".format(predict_count)
print "Over range Count: {0}".format(orc)

In [ ]:
# http://stackoverflow.com/questions/17197492/root-mean-square-error-in-python
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [1]:
# now we need to do the comparisons to check how well our imputation worked!
# to test / grade our efforts, we will use r^2 correlation between this vector + the actual "ground truth" vector
# We will also use RMSE value, and a graph of the residuals to understand where the error is most concentrated.
# Due to a lack of time, we will not be able to iterate on this design; however, hopefully this will at the very
# least provide good insight into how we could progress further if we wanted to.
%matplotlib inline
import matplotlib.pyplot as plt

full_values_vector = sample_f_bed[4]
full_values = [full_values_vector[x] for x in range(len(filled_test_tissue)) if ~np.isnan(full_values_vector[x])]
filled_test = [filled_test_tissue[x] for x in range(len(filled_test_tissue)) if ~np.isnan(full_values_vector[x])]
diff_test = [filled_test[x] - full_values[x] for x in range(len(filled_test))]
print len(full_values)
print len(filled_test)
print(np.corrcoef(filled_test, full_values))
print(rmse(np.array(filled_test), np.array(full_values)))
plt.figure(1)
plt.scatter(np.array(full_values), np.array(filled_test))
plt.figure(2)
plt.scatter([i for i in range(len(filled_test))], diff_test)
plt.show()

NameError: name 'sample_f_bed' is not defined

In [98]:
for x in filled_test:
#     print x
    assert -1 <= x <= 1, x

AssertionError: -1.79642610239